In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import shutil

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
print(tf.__version__)

# Any results you write to the current directory are saved as output.

In [ ]:
df = pd.read_csv('../input/kiva_loans.csv')
df[:3]

In [ ]:
x = tf.constant(df['funded_amount'])
y = tf.constant(df['loan_amount'])
z = tf.add(x, y)
print(z)

with tf.Session() as sess:
    r = sess.run(z)
    print(r)

In [ ]:
from sklearn.model_selection import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(df[['loan_amount', 'sector', 'activity', 'date']], df['funded_amount'], test_size = 0.3)
yTrain = pd.DataFrame(yTrain)
yTest = pd.DataFrame(yTest)

xTrain, xEval, yTrain, yEval = train_test_split(xTrain['loan_amount'], yTrain['funded_amount'], test_size = 0.3)
xTrain = pd.DataFrame(xTrain)
xEval = pd.DataFrame(xEval)
yTrain = pd.DataFrame(yTrain)
yEval = pd.DataFrame(yEval)

In [ ]:
shutil.rmtree("outdir", ignore_errors = True)

featureCols = [
    tf.feature_column.numeric_column('loan_amount')
]

linModel = tf.estimator.LinearRegressor(featureCols)

def train_input_fn(df_fn_x, df_fn_y):
    return tf.estimator.inputs.pandas_input_fn(
        x = df_fn_x,
        y = df_fn_y['funded_amount'],
        batch_size = 128,
        num_epochs = 3,
        shuffle = True,
        queue_capacity = 1000
    )

def evaluate_input_fn(df_fn_x, df_fn_y):
    return tf.estimator.inputs.pandas_input_fn(
        x = df_fn_x,
        y = df_fn_y['funded_amount'],
        batch_size = 128,
        shuffle = False,
        queue_capacity = 1000
    )

def predict_input_fn(df_fn_x, df_fn_y):
    return tf.estimator.inputs.pandas_input_fn(
        x = df_fn_x,
        y = None,
        batch_size = 128,
        shuffle = False,
        queue_capacity = 1000
    )

linModel.train(train_input_fn(xTrain, yTrain))
metrics = linModel.evaluate(evaluate_input_fn(xEval, yEval))
predictions = linModel.predict(predict_input_fn(xTest, yTest))

In [ ]:
np.sqrt(metrics['average_loss'])